# Анализ эксперимента от 20.10.25 c изменением коэффициента Kp посредством приращений

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path

EXPERIMENT_NAME = "kp_scan"
EXPERIMENT_DATE = "2025-10-20"
EXPERIMENT_TIME = "22-00-22"

PATH_TO_EXP_DIR = Path(f"../experiments/{EXPERIMENT_NAME}/{EXPERIMENT_DATE}_{EXPERIMENT_TIME}")
LOG_DIR = PATH_TO_EXP_DIR / "logs"

In [ ]:
import pandas as pd
import re

def parse(file_path):
    with open(file_path, "r") as f:
        lines = f.readlines()

    send_pattern = re.compile(
        r"step=(?P<step>\d+)\s+time=(?P<time>[0-9.]+)\s+send\s+"
        r"kp=(?P<kp>[0-9.\-]+)\s+ki=(?P<ki>[0-9.\-]+)\s+kd=(?P<kd>[0-9.\-]+)\s+u_min=(?P<u_min>[0-9.\-]+)\s+u_max=(?P<u_max>[0-9.\-]+)"
    )
    recv_pattern = re.compile(
        r"step=(?P<step>\d+)\s+time=(?P<time>[0-9.]+)\s+recv\s+"
        r"process_variable=(?P<process_variable>[0-9.\-]+)\s+control_output=(?P<control_output>[0-9.\-]+)"
    )

    sends, recvs = {}, {}

    for line in lines:
        s = send_pattern.search(line)
        if s:
            d = s.groupdict()
            step = int(d.pop("step"))
            d = {k: float(v) for k, v in d.items()}
            sends[step] = d
            continue

        r = recv_pattern.search(line)
        if r:
            d = r.groupdict()
            step = int(d.pop("step"))
            d = {k: float(v) for k, v in d.items()}
            recvs[step] = d

    records = []
    for i, step in enumerate(sorted(set(sends.keys()) | set(recvs.keys()))):
        data = {"step": i}
        if step in sends:
            data.update(sends[step])
        if step in recvs:
            data.update(recvs[step])
        records.append(data)

    df = pd.DataFrame(records)
    return df

In [ ]:
data_df = parse(LOG_DIR / "log.txt")

In [ ]:
data_df.info()

In [ ]:
data_df.head()

In [ ]:
data_df.tail()

## Этап warmup

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

WARMUP_STEPS = 1000
warmup_df = data_df[data_df["step"] < WARMUP_STEPS]

SETPOINT = 1200

fig, axes = plt.subplots(2, 1, figsize=(10, 8), sharex=True)

sns.lineplot(ax=axes[0], x="step", y="process_variable", data=warmup_df)
axes[0].axhline(SETPOINT, color="red", linestyle="--", label=f"SETPOINT = {SETPOINT}")
axes[0].set_title("Process Variable (Warmup)")
axes[0].set_ylabel("process_variable")
axes[0].grid(True)
axes[0].legend()

sns.lineplot(ax=axes[1], x="step", y="control_output", data=warmup_df)
axes[1].set_title("Control Output (Warmup)")
axes[1].set_xlabel("step")
axes[1].set_ylabel("control_output")
axes[1].grid(True)

plt.tight_layout()
plt.show()

## Этап перебора коэффициента Kp

In [ ]:
KP_SCAN_ASCENDING_STEPS = 100 * 200
WARMUP_STEPS = 1000

kp_scan_ascending_df = data_df[
    (data_df["step"] > WARMUP_STEPS) &
    (data_df["step"] < WARMUP_STEPS + KP_SCAN_ASCENDING_STEPS)
]
kp_scan_ascending_df["step"] = kp_scan_ascending_df["step"] - WARMUP_STEPS

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(10, 8), sharex=True)

sns.lineplot(ax=axes[0], x="step", y="process_variable", data=kp_scan_ascending_df)
axes[0].axhline(SETPOINT, color="red", linestyle="--", label=f"SETPOINT = {SETPOINT}")
axes[0].set_title("Process Variable")
axes[0].set_ylabel("process_variable")
axes[0].grid(True)
axes[0].legend()

sns.lineplot(ax=axes[1], x="step", y="control_output", data=kp_scan_ascending_df)
axes[1].set_title("Control Output")
axes[1].set_xlabel("")
axes[1].set_ylabel("control_output")
axes[1].grid(True)

sns.lineplot(ax=axes[2], x="step", y="kp", data=kp_scan_ascending_df)
axes[2].set_title("Kp")
axes[2].set_xlabel("step")
axes[2].set_ylabel("Kp")
axes[2].grid(True)

plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

BLOCK_SIZE = 200
LAST_POINTS = 100
SETPOINT = 1200

error_means = []
error_stds = []
kp_values = []

total_steps = kp_scan_ascending_df.shape[0]
num_blocks = total_steps // BLOCK_SIZE

for i in range(num_blocks):
    block = kp_scan_ascending_df.iloc[i*BLOCK_SIZE:(i+1)*BLOCK_SIZE]
    last_points = block.tail(LAST_POINTS)
    error = last_points["process_variable"] - SETPOINT
    error_means.append(error.mean())
    error_stds.append(error.std())
    kp_values.append(block["kp"].iloc[-1])

fig, axes = plt.subplots(3, 1, figsize=(10, 8), sharex=True)

axes[0].plot(range(num_blocks), error_means)
axes[0].set_title("Error Mean per Block")
axes[0].set_ylabel("Error Mean")
axes[0].grid(True)

axes[1].plot(range(num_blocks), error_stds, color='orange')
axes[1].set_title("Error Std per Block")
axes[1].set_ylabel("Error Std")
axes[1].grid(True)

axes[2].plot(range(num_blocks), kp_values, color='green')
axes[2].set_title("Kp per Block")
axes[2].set_xlabel("Block Index")
axes[2].set_ylabel("Kp")
axes[2].grid(True)

plt.tight_layout()
plt.show()

## Этап перебора коэффицента Kp с нисхождением

In [ ]:
KP_SCAN_DESCENDING_STEPS = 100 * 200
WARMUP_STEPS = 1000

kp_scan_descending_df = data_df[
    (data_df["step"] > WARMUP_STEPS + KP_SCAN_ASCENDING_STEPS) &
    (data_df["step"] < WARMUP_STEPS + KP_SCAN_ASCENDING_STEPS + KP_SCAN_DESCENDING_STEPS)
]
kp_scan_descending_df["step"] = kp_scan_descending_df["step"] - (WARMUP_STEPS + KP_SCAN_ASCENDING_STEPS)

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(10, 8), sharex=True)

sns.lineplot(ax=axes[0], x="step", y="process_variable", data=kp_scan_descending_df)
axes[0].axhline(SETPOINT, color="red", linestyle="--", label=f"SETPOINT = {SETPOINT}")
axes[0].set_title("Process Variable")
axes[0].set_ylabel("process_variable")
axes[0].grid(True)
axes[0].legend()

sns.lineplot(ax=axes[1], x="step", y="control_output", data=kp_scan_descending_df)
axes[1].set_title("Control Output")
axes[1].set_xlabel("")
axes[1].set_ylabel("control_output")
axes[1].grid(True)

sns.lineplot(ax=axes[2], x="step", y="kp", data=kp_scan_descending_df)
axes[2].set_title("Kp")
axes[2].set_xlabel("step")
axes[2].set_ylabel("Kp")
axes[2].grid(True)

plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

BLOCK_SIZE = 200
LAST_POINTS = 100
SETPOINT = 1200

error_means = []
error_stds = []
kp_values = []

total_steps = kp_scan_descending_df.shape[0]
num_blocks = total_steps // BLOCK_SIZE

for i in range(num_blocks):
    block = kp_scan_descending_df.iloc[i*BLOCK_SIZE:(i+1)*BLOCK_SIZE]
    last_points = block.tail(LAST_POINTS)
    error = last_points["process_variable"] - SETPOINT
    error_means.append(error.mean())
    error_stds.append(error.std())
    kp_values.append(block["kp"].iloc[-1])

fig, axes = plt.subplots(3, 1, figsize=(10, 8), sharex=True)

axes[0].plot(range(num_blocks), error_means)
axes[0].set_title("Error Mean per Block")
axes[0].set_ylabel("Error Mean")
axes[0].grid(True)

axes[1].plot(range(num_blocks), error_stds, color='orange')
axes[1].set_title("Error Std per Block")
axes[1].set_ylabel("Error Std")
axes[1].grid(True)

axes[2].plot(range(num_blocks), kp_values, color='green')
axes[2].set_title("Kp per Block")
axes[2].set_xlabel("Block Index")
axes[2].set_ylabel("Kp")
axes[2].grid(True)

plt.tight_layout()
plt.show()

## Винеровский процесс

In [ ]:
WARMUP_STEPS = 1000
KP_SCAN_ASCENDING_STEPS = 100 * 200
KP_SCAN_DESCENDING_STEPS = 100 * 200
WINNER_STEPS = 100_000

winner_df = data_df[
    (data_df["step"] > 2 * WARMUP_STEPS + KP_SCAN_ASCENDING_STEPS + KP_SCAN_DESCENDING_STEPS) &
    (data_df["step"] < 2 * WARMUP_STEPS + KP_SCAN_ASCENDING_STEPS + KP_SCAN_DESCENDING_STEPS + WINNER_STEPS)
].copy()

winner_df["step"] = winner_df["step"] - (2 * WARMUP_STEPS + KP_SCAN_ASCENDING_STEPS + KP_SCAN_DESCENDING_STEPS)


In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(10, 8), sharex=True)

sns.lineplot(ax=axes[0], x="step", y="process_variable", data=winner_df)
axes[0].axhline(SETPOINT, color="red", linestyle="--", label=f"SETPOINT = {SETPOINT}")
axes[0].set_title("Process Variable")
axes[0].set_ylabel("process_variable")
axes[0].grid(True)
axes[0].legend()

sns.lineplot(ax=axes[1], x="step", y="control_output", data=winner_df)
axes[1].set_title("Control Output")
axes[1].set_xlabel("")
axes[1].set_ylabel("control_output")
axes[1].grid(True)

sns.lineplot(ax=axes[2], x="step", y="kp", data=winner_df)
axes[2].set_title("Kp")
axes[2].set_xlabel("step")
axes[2].set_ylabel("Kp")
axes[2].grid(True)

plt.tight_layout()
plt.show()